In [1]:
import argparse
import os
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from jax.numpy.fft import irfft, rfft, fft, ifft

from jax import grad, jit, vmap
import jax.numpy as jnp
import jax.random as random
import jax

!pip install numpyro
import numpyro
from numpyro import handlers
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import numpyro.contrib.module as module

!pip install -q flax
from flax import linen as nn

import torch
from torchvision import transforms, datasets



matplotlib.use("Agg")  # noqa: E402
np.random.seed(0)
key = numpyro.prng_key()
numpyro.set_host_device_count(4)

/home/studio-lab-user/.conda/envs/jax/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(jax.devices(backend='gpu'))

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]


In [3]:
def numpy_collate(batch):
  if isinstance(batch[0], np.ndarray):
    return np.stack(batch)
  elif isinstance(batch[0], (tuple,list)):
    transposed = zip(*batch)
    return [numpy_collate(samples) for samples in transposed]
  else:
    return np.array(batch)


class NumpyLoader(torch.utils.data.DataLoader):
  def __init__(self, dataset, batch_size=1,
                shuffle=False, sampler=None,
                batch_sampler=None, num_workers=0,
                pin_memory=False, drop_last=False,
                timeout=0, worker_init_fn=None):
    super(self.__class__, self).__init__(dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        sampler=sampler,
        batch_sampler=batch_sampler,
        num_workers=num_workers,
        collate_fn=numpy_collate,
        pin_memory=pin_memory,
        drop_last=drop_last,
        timeout=timeout,
        worker_init_fn=worker_init_fn)

class FlattenAndCast(object):
  def __call__(self, pic):
    return np.ravel(np.array(pic, dtype=jnp.float32))

In [4]:
def get_mnist(n, m):
    """
    Download MNIST and return train and evaluation sets.
    """
    mnist = datasets.MNIST('data',
        train=True,
        download=True,
        transform=FlattenAndCast())
    mnist = list(mnist)
    # One batch with all of mnist
    train_loader = NumpyLoader(mnist, batch_size=len(mnist), num_workers=0)
    x, y = list(train_loader)[0]
    # Normalize
    x = (x - x.mean()) / x.std()
    # Train and test set
    train_x, train_y = x[0:n], y[0:n]
    val_x, val_y = x[n:n+m], y[n:n+m]
    return train_x, train_y, val_x, val_y

In [5]:
class flax_CNN(nn.Module):
    @nn.compact   
    def __call__(self, x):
        #print(x.shape)
        x_length = x.shape[0]
        x = nn.Conv(features = 8, kernel_size = (5,5),strides = (1,1), 
                   padding = (2,2), 
                   use_bias= False)(x)
        print(x.shape)
        x = nn.max_pool(x,window_shape=(2,2),strides = (2,2))
        all_len = len(x.flatten())
        x = x.reshape((x_length, int((all_len/x_length))))

        return x


In [6]:
_x, _y, _xv, _yv = get_mnist(50000,10000)

# Training
N=1000 
# Test
M=300

# Get the training and test data from the MNIST global variables
x, y, xv, yv = _x[0:N], _y[0:N], _xv[N:N+M], _yv[N:N+M]


In [7]:
dim1 = 128
dim2 = 32
key = random.PRNGKey(0)
conv = flax_CNN() #from (200,28,28) to (200, 1568)

rng_key = random.PRNGKey(1)

print(conv.tabulate(jax.random.PRNGKey(0), x.reshape((-1,28,28,1))))


(1000, 28, 28, 8)

                                flax_CNN Summary                                
┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ path   ┃ module   ┃ inputs            ┃ outputs           ┃ params           ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│        │ flax_CNN │ float32[1000,28,… │ float32[1000,156… │                  │
├────────┼──────────┼───────────────────┼───────────────────┼──────────────────┤
│ Conv_0 │ Conv     │ float32[1000,28,… │ float32[1000,28,… │ kernel:          │
│        │          │                   │                   │ float32[5,5,1,8] │
│        │          │                   │                   │                  │
│        │          │                   │                   │ 200 (800 B)      │
├────────┼──────────┼───────────────────┼───────────────────┼──────────────────┤
│        │          │                   │             Total │ 200 (800 B)      │
└────────

In [59]:
def circ_matmul(x, w, output_size):
    # xw = fft(fft(w)*ifft(x)).real
    # Note the use of the n argument to get right output shape
    xw = irfft(jnp.conj(rfft(w)) * rfft(x), output_size)
    return xw

In [9]:
# the non-linearity we use in our neural network
def nonlin(x):
    return jnp.tanh(x)

In [55]:
def circ_conv(x, w,input_size, output_size):
    out = irfft(jnp.conj(rfft(w)) * rfft(x), n=output_size)
    print(out.shape)
    return out
    

In [57]:
out = circ_conv(x[0], jnp.ones([1, 784]), 784, 1586)
print(out)

(1, 1586)
[[8.829421 8.829421 8.829421 ... 8.829421 8.829421 8.829421]]


In [60]:
# TODO: add numlayer parameter, add whether has cnn parameter, add whether use circulant multiply, add cnn parameters.calculate dimensions. 
print(x.shape)
def model_circulant_weight(x, y=None, dim1 = dim1, dim2 = dim2):
    w1 = numpyro.sample("w1", dist.Normal(0,1).expand([28*28*2]).to_event(1))
    b1 = numpyro.sample("b1", dist.Normal(0,1).expand([dim1]).to_event(1))

    w2 = numpyro.sample("w2", dist.Normal(0,1).expand([dim1]).to_event(1))
    b2 = numpyro.sample("b2", dist.Normal(0,1).expand([dim2]).to_event(1))


    w3 = numpyro.sample("w3", dist.Normal(0,1).expand([dim2,10]).to_event(2))
    b3 = numpyro.sample("b3", dist.Normal(0,1).expand([10]).to_event(1))    

    # Convolution
    # conv_numpyro = module.random_flax_module("conv", conv, dist.Normal(0, 1), input_shape=((x.shape[0],28,28,1)))
    # cx = nonlin(conv_numpyro(x.reshape((-1,28,28,1))))
    
    w_conv = numpyro.sample("w_conv", dist.Normal(0,1).expand([28*28]))
    cx = nonlin(circ_matmul(x, w_conv, 28*28*2))
    #print(shape)

    # Layer 1: dim1
    
    h1 = circ_matmul(cx, w1, dim1)
    h1 = nonlin(h1[:, 0:dim1] + b1)
    # Layer 2: dim2
    h2 = circ_matmul(h1, w2, dim2)
    h2 = nonlin(h2[:, 0:dim2] + b2)

    # Layer 3: dim=10 (logits)
    h3 = jnp.matmul(h2,w3) + b3
    # Register the logits for easy prediction
    numpyro.deterministic("logits", h3)

    # Likelihood
    with numpyro.plate("labels", x.shape[0]):
        y_obs = numpyro.sample("y_obs", dist.CategoricalLogits(logits=h3), 
                               obs=y, rng_key=key)

(1000, 784)


In [31]:
def model_full_weight(x, y=None):
  w1_full = numpyro.sample("w1_full", dist.Normal(0,1).expand([2*28*28,dim1]).to_event(1)) #weight matrix dimension: (indim(1568), h1dim(dim1 128))
  b1_full = numpyro.sample("b1_full", dist.Normal(0,1).expand([dim1]).to_event(1)) # bias dimension: (128,)

  w2_full = numpyro.sample("w2_full", dist.Normal(0,1).expand([dim1,dim2]).to_event(1)) #weight matrix dimension: (h1dim(dim1), h2dim(dim2))
  b2_full = numpyro.sample("b2_full", dist.Normal(0,1).expand([dim2]).to_event(1)) # bias dimension: (dim2,)

  w3_full = numpyro.sample("w3_full", dist.Normal(0,1).expand([dim2,10]).to_event(2)) #weight matrix dimension: (h2dim(dim2), outdim(10))
  b3_full = numpyro.sample("b3_full", dist.Normal(0,1).expand([10]).to_event(1)) # bias dimension: (outdim(10))  

  conv_numpyro = module.random_flax_module("conv_full", conv, dist.Normal(0, 1), input_shape=((x.shape[0],28,28,1)))
  cx = nonlin(conv_numpyro(x.reshape((-1,28,28,1))))


  h1 = nonlin(jnp.matmul(cx,w1_full) + b1_full)

  h2 = nonlin(jnp.matmul(h1,w2_full) + b2_full)

  h3 = jnp.matmul(h2,w3_full) + b3_full

  # Register the logits for easy prediction
  numpyro.deterministic("logits_full", h3)


  # Likelihood
  with numpyro.plate("labels_full", x.shape[0]):
      y_obs_f = numpyro.sample("y_obs_full", dist.CategoricalLogits(logits = h3), obs = y, rng_key=key)




In [32]:
# The point estimate model common neural network

class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = torch.nn.Conv2d(
                in_channels=1,
                out_channels=8,
                kernel_size=5,
                stride=1,
                padding=2,
                bias = False
            )
        # Lift to Pyro
        self.maxp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Sequential(torch.nn.Linear(in_features=2*28*28, out_features=dim1), torch.nn.Tanh(), 
                                      torch.nn.Linear(in_features= dim1, out_features=dim2), torch.nn.Tanh(), 
                                      torch.nn.Linear(in_features = dim2, out_features=10), torch.nn.Tanh())

    def forward(self, x):
        cx = self.conv(x)
        px = self.maxp(cx)
        fx = torch.flatten(px, 1)
        x_output = self.fc(fx)
        return x_output


In [33]:
jax.local_device_count()

1

In [61]:
# train the circulant matrix model
%time
kernel = NUTS(model_circulant_weight,
              target_accept_prob = 0.8,
              max_tree_depth = 10
              )

mcmc = MCMC(kernel,
            num_samples = 100,
            num_warmup = 50,
            num_chains = 2,
            progress_bar = True)

mcmc.run(random.PRNGKey(0), x,y)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 14.1 µs


/tmp/ipykernel_243/3640374796.py:8: UserWarning: There are not enough devices to run parallel chains: expected 2 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(2)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC(kernel,
sample: 100%|██████████| 150/150 [05:02<00:00,  2.02s/it, 1023 steps of size 1.07e-03. acc. prob=0.98]


In [ ]:
# Train the full weight matrix model
kernel_1 = NUTS(model_full_weight,
              target_accept_prob = 0.8,
              max_tree_depth = 10
              )

mcmc_1 = MCMC(kernel_1,
            num_samples = 100,
            num_warmup = 50,
            num_chains = 2,
            progress_bar = True)

mcmc_1.run(random.PRNGKey(1), x, y)

/tmp/ipykernel_243/4160828757.py:7: UserWarning: There are not enough devices to run parallel chains: expected 2 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(2)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc_1 = MCMC(kernel_1,


(1000, 28, 28, 8)
(1000, 28, 28, 8)
(1000, 28, 28, 8)


  0%|          | 0/150 [00:00<?, ?it/s]

(1000, 28, 28, 8)


sample: 100%|██████████| 150/150 [07:36<00:00,  3.04s/it, 1023 steps of size 2.68e-03. acc. prob=0.91]


(1000, 28, 28, 8)
(1000, 28, 28, 8)
(1000, 28, 28, 8)
(1000, 28, 28, 8)


sample: 100%|██████████| 150/150 [07:33<00:00,  3.02s/it, 1023 steps of size 2.31e-03. acc. prob=0.92]


In [39]:
# Train the point estimation mmodel

def get_mnist_torch(n, m):
    """
    Download MNIST and return train and evaluation sets.
    """
    img_to_tensor = transforms.ToTensor()
    mnist = datasets.MNIST('data',
        train=True,
        download=True,
        transform=img_to_tensor)
    mnist = list(mnist)
    # One batch with all of mnist
    train_loader = torch.utils.data.DataLoader(mnist,
        batch_size=len(mnist),
        shuffle=True)
    # x = images tensor, y = labels tensor 
    x, y = list(train_loader)[0]
    # Flatten images
    x = x.view(-1, 28*28)
    # Normalize
    x = (x - x.mean()) / x.std()
    # Train and test set
    train_x, train_y = x[0:n], y[0:n]
    val_x, val_y = x[n:n+m], y[n:n+m]
    return train_x, train_y, val_x, val_y


def train_point_estimation(model, dataloader, criterion, optimizer, num_epochs):
    model.cuda()
    loss_list = []
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for _, (data, label) in enumerate(dataloader):

            data = data.cuda()
            labels = label.cuda()
            model.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        # Print the average loss at every epoch
        average_loss = epoch_loss / (len(dataloader))
        loss_list.append(average_loss)
        if (epoch + 1) % 10 == 0:
            print(f'Epoch: {epoch + 1}, Average Loss: {average_loss}')
    return loss_list

point_estimate_model = CNN()
x_torch, y_torch, xv_torch, yv_torch = get_mnist_torch(N,M)

x_torch = x_torch.view((-1,1,28,28))
print(x_torch.shape)
xv_torch = xv_torch.view((-1,1,28,28))


train_set = torch.utils.data.TensorDataset(x_torch,y_torch)
test_set = torch.utils.data.TensorDataset(xv_torch,yv_torch)

train_loader = torch.utils.data.DataLoader(train_set, shuffle = True, batch_size = 1)

test_loader = torch.utils.data.DataLoader(test_set, shuffle = True, batch_size = 1)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(point_estimate_model.parameters(), lr = 0.001)

loss_list_point_estimation = train_point_estimation(point_estimate_model,train_loader, criterion, optimizer, 100)


    

torch.Size([1000, 1, 28, 28])
Epoch: 10, Average Loss: 0.8373497306108475
Epoch: 20, Average Loss: 0.8354415633082389
Epoch: 30, Average Loss: 0.8034206417202949
Epoch: 40, Average Loss: 0.8174283702969551
Epoch: 50, Average Loss: 0.8096831457614899
Epoch: 60, Average Loss: 0.8151490816473961
Epoch: 70, Average Loss: 0.7996587716937065
Epoch: 80, Average Loss: 0.8067343647480011
Epoch: 90, Average Loss: 0.8071548728346825
Epoch: 100, Average Loss: 0.8047575291991234


In [68]:
# get posterior and prediction of the circular weight matrix model

posterior_samples = mcmc.get_samples()


posterior_predictive_test = numpyro.infer.Predictive(model_circulant_weight, posterior_samples)(
        jax.random.PRNGKey(3),xv)

posterior_predictive_train = numpyro.infer.Predictive(model_circulant_weight, posterior_samples)(
        jax.random.PRNGKey(3),x)

prior_predictive = numpyro.infer.Predictive(model_circulant_weight, num_samples=500)(
        jax.random.PRNGKey(3),xv)


In [41]:
# get posterior and prediction of the full weight matrix model

posterior_samples_1 = mcmc_1.get_samples()


posterior_predictive_test_1 = numpyro.infer.Predictive(model_full_weight, posterior_samples_1)(
        jax.random.PRNGKey(3),xv)

posterior_predictive_train_1 = numpyro.infer.Predictive(model_full_weight, posterior_samples_1)(
        jax.random.PRNGKey(3),x)

prior_predictive_1 = numpyro.infer.Predictive(model_full_weight, num_samples=500)(
        jax.random.PRNGKey(3),xv)

(300, 28, 28, 8)
(300, 28, 28, 8)
(1000, 28, 28, 8)
(1000, 28, 28, 8)
(300, 28, 28, 8)
(300, 28, 28, 8)


In [64]:
print(posterior_samples.keys())
print(posterior_samples_1.keys())
print(posterior_samples['w_conv'].shape)
print(posterior_samples_1['conv_full/Conv_0.kernel'].shape)
print(posterior_samples['w1'].shape)
print(posterior_samples_1['w1_full'].shape)
print(len(posterior_predictive_test_1.keys()))

dict_keys(['b1', 'b2', 'b3', 'logits', 'w1', 'w2', 'w3', 'w_conv'])
dict_keys(['b1_full', 'b2_full', 'b3_full', 'conv_full/Conv_0.kernel', 'logits_full', 'w1_full', 'w2_full', 'w3_full'])
(200, 784)
(200, 5, 5, 1, 8)
(200, 1568)
(200, 1568, 128)
2


In [43]:
!pip install arviz
import arviz as az
az.style.use("arviz-doc")

In [44]:
def accuracy(pred, data):
    """
    Calculate accuracy of predicted labels (integers).

    pred: predictions, ndarray[sample_index, chain_index, data_index, logits]
    data: actual data (digit), ndarray[data_index]

    Prediction is taken as most common predicted value.
    Returns accuracy (#correct/#total).
    """
    n=data.shape[0]
    correct=0
    total=0
    for i in range(0, n):
        # Get most common prediction value from logits
        pred_i=int(jnp.argmax(jnp.sum(pred[:,i,:],0)))
        # Compare prediction with data
        if int(data[i])==int(pred_i):
            correct+=1.0
        total+=1.0
    # Return fractional accuracy
    return correct/total

In [65]:
# summary of circulant matrix model


#summary_data_circulant = arviz.from_numpyro(posterior=mcmc, prior=prior_predictive, posterior_predictive= posterior_predictive_test )
summary_data_circulant = az.convert_to_inference_data(posterior_samples)
az.plot_ess(summary_data_circulant,var_names=['w1'], kind = 'evolution')
plt.savefig("posterior_sample.png")

/home/studio-lab-user/.conda/envs/jax/lib/python3.9/site-packages/arviz/data/base.py:221: UserWarning: More chains (200) than draws (128). Passed array should have shape (chains, draws, *shape)
  warnings.warn(
/home/studio-lab-user/.conda/envs/jax/lib/python3.9/site-packages/arviz/data/base.py:221: UserWarning: More chains (200) than draws (32). Passed array should have shape (chains, draws, *shape)
  warnings.warn(
/home/studio-lab-user/.conda/envs/jax/lib/python3.9/site-packages/arviz/data/base.py:221: UserWarning: More chains (200) than draws (10). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


In [69]:
# Accuracy on test set
logits = posterior_predictive_test['logits']
print("Success posterior test = %.3f" % accuracy(logits, yv))

# Accuracy on training set
logits = posterior_predictive_train['logits']
print("Success posterior training = %.3f" % accuracy(logits, y))

logits = prior_predictive['logits']
print("Success prior = %.3f" % accuracy(logits, yv))

print("Posterior test diagnostics:")
numpyro.diagnostics.print_summary(posterior_samples)

Success posterior test = 0.557
Success posterior training = 0.921
Success prior = 0.090
Posterior test diagnostics:

                mean       std    median      5.0%     95.0%     n_eff     r_hat
        b1      0.10      1.09      0.13     -1.64      1.89  20719.68      0.99
        b2      0.24      1.12      0.25     -1.58      2.09   7430.46      0.97
        b3     -0.13      1.20     -0.04     -1.74      1.74  -9494.41      0.93
 logits[0]      0.40      4.21      0.67     -6.65      6.88  13484.60      1.02
 logits[1]      0.07      4.16      0.05     -6.87      6.72  60594.09      1.02
 logits[2]      0.27      5.07      0.28     -8.05      8.85  66709.05      1.01
 logits[3]      0.34      3.81      0.18     -5.77      6.78   7614.71      1.03
 logits[4]     -0.27      4.48     -0.33     -7.41      7.26 187372.40      1.01
 logits[5]      1.43      3.19      1.39     -3.70      6.62   3362.10      1.04
 logits[6]     -1.48      5.79     -1.19    -11.50      7.82 186262.22   

In [67]:
# summary of full weight matrix model



# Accuracy on test set
logits = posterior_predictive_test_1['logits_full']

print("Success posterior test = %.3f" % accuracy(logits, yv))

# Accuracy on training set
logits = posterior_predictive_train_1['logits_full']
print("Success posterior training = %.3f" % accuracy(logits, y))

logits = prior_predictive_1['logits_full']
print("Success prior = %.3f" % accuracy(logits, yv))

print("Posterior test diagnostics:")
numpyro.diagnostics.print_summary(posterior_samples_1)

Success posterior test = 0.923
Success posterior training = 1.000
Success prior = 0.117
Posterior test diagnostics:

                                    mean       std    median      5.0%     95.0%     n_eff     r_hat
                       b1_full     -0.02      1.00     -0.02     -1.64      1.63  25890.54      1.00
                       b2_full     -0.04      0.96     -0.02     -1.66      1.52   6834.92      1.00
                       b3_full     -0.07      0.94     -0.10     -1.51      1.54   2461.68      1.01
conv_full/Conv_0.kernel[0,0,0]     -0.33      0.96     -0.30     -1.81      1.30   7014.18      0.97
conv_full/Conv_0.kernel[0,0,1]     -0.04      0.87     -0.06     -1.52      1.35  -7161.55      0.85
conv_full/Conv_0.kernel[0,0,2]      0.16      1.09      0.16     -1.62      1.90   1153.89      1.24
conv_full/Conv_0.kernel[0,0,3]     -0.34      0.84     -0.35     -1.81      0.87   3315.58      0.86
conv_full/Conv_0.kernel[0,0,4]     -0.21      0.90     -0.22     -1.67     

In [ ]:
#